# Evaluate Pseudo Labels

Questions:
1. Do labels obtained from semisupervised classifier improve a supervised classifier performance?
2. Do high confident labels from a SSL model improve a supervised classifier performance?

## Design

Original data set has 2801 samples x 5072 features. This data were split into 70% train and 30% test. This process was repeated 7 times to create 7 different train-test splits.

For each split:
1. Create a random forest model
2. Evaluate the model on 35% data with itest
3. Evaluate the model on 35% data  + SSL GSE109379 with itest
4. Evaluate the model on 35% data + 35% pseudo-label data + GSE109379 pseudo-label data with itest
5. Evaluate the model on 35% data + 35% pseudo-label data + HC GSE1093790 pseudo-label data with itest
6. Evaluate the model on 35% data + high confident pseudo-label +  GSE109379 pseudo-label data with itest
7. Evaluate the model on 35% data + high confident pseudo-label +  HC GSE109379 pseudo-label data with itest


Box plots for each model (1 box plot show the average accuracies of each model over the 7 train-test splits)

In [193]:
import scipy.io
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

import pandas as pd
import numpy as np
import collections

import tensorflow as tf
from tensorflow import keras

from tensorflow import feature_column
from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import RepeatedKFold

def CountFrequency(arr):
    return collections.Counter(arr)  

def combine(y, str):
    freq = CountFrequency(y)
    d = pd.DataFrame(freq.items(), columns=['Sample', 'pseudo_class_label'] )    
    schema = [str] * len(d)
    d['Dataset'] = schema
    return d

def get_freq_series(y):
    freq = CountFrequency(y)
    s = pd.Series(freq.values(), index = freq.keys())
    return s

def process_csv(filename):
    df = pd.read_csv(filename, index_col=0)
    y = df.y
    X = df.drop(columns=['y'])
    return (X, y)


def process_csv_encoder(filename):
    label_encoder = LabelEncoder()
    df = pd.read_csv(filename, index_col=0)    
    y = label_encoder.fit_transform(df.y)
    X = df.drop(columns=['y'])
    return(X, y)


def process_csv_oneHotEncoder(filename):
    df = pd.read_csv(filename, index_col=0)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(df.y)
    # one hot encode
    encoded = keras.utils.to_categorical(integer_encoded)
    X = df.drop(columns=['y'])
    return(X, encoded)

def append_result(p, df):
    s = pd.Series(p, index=df.columns)
    return df.append(s, ignore_index=True)

def apply_CV(mod, cv, X, y):
    return cross_validate(mod, X, y, cv=cv, scoring=['balanced_accuracy', 'accuracy', 'recall_weighted'])

def append_cv_results(df, r, seed_name, dataset_name):
    df = append_result([seed_name, dataset_name, 'cross_val', 'balanced_acc', r['test_balanced_accuracy'].mean().round(2)], df)
    df = append_result([seed_name, dataset_name, 'cross_val', 'accuracy', r['test_accuracy'].mean().round(2)], df)
    df = append_result([seed_name, dataset_name, 'cross_val', 'recall_weighted', r['test_recall_weighted'].mean().round(2)], df)
    return df

def accuracy_per_class(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    classes = np.unique(y_true)
    per_class_accuracies = cm.diagonal()/cm.sum(axis=1)
    per_class_acc_wKeys = {}
    for idx, cls in enumerate(classes):
        per_class_acc_wKeys[cls] = per_class_accuracies[idx]
    return per_class_acc_wKeys

def evaluate_against_test_set(mod, x_train, y_train, x_test, y_test):
    mod.fit(x_train, y_train)
    y_pred = mod.predict(x_test)
    per_class_acc = accuracy_per_class(y_test, y_pred)
    rec = recall_score(y_test, y_pred,  average='weighted', zero_division=0).round(3)
    acc = accuracy_score(y_pred, y_test).round(3)
    b_acc = balanced_accuracy_score(y_pred, y_test).round(3)
    return (rec, acc, b_acc, per_class_acc)

def append_testset_results(df, r, seed_name, dataset_name):
    df = append_result([seed_name, dataset_name, 'vs_testset', 'accuracy', r[0]], df)
    df = append_result([seed_name, dataset_name, 'vs_testset', 'balanced_acc', r[1]], df)
    df = append_result([seed_name, dataset_name, 'vs_testset', 'recall_weighted', r[2]], df)
    return df


def create_per_class_acc_df(class_acc, seed, type_dset):
    d = pd.DataFrame(columns = ['Seed', 'Dataset'])
    d = d.append(class_acc, ignore_index=True)
    d['Seed'] = seed
    d['Dataset'] = type_dset
    return d

def create_model(n_inputs):
    model = keras.Sequential()
    model.add(keras.layers.Dense(1000, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(500, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(75, activation="softmax"))
    
    model.compile(optimizer = 'adam', loss= 'categorical_crossentropy', 
                  metrics=['accuracy', 'Precision','Recall', 'mae', 'mse'])
    
    return model

# evaluate a model with a given number of repeats
def evaluate_model(mod, X, y, repeats):
    # prepare the repeated stratified cross-validation procedure
    kfold = RepeatStratifiedKFold(n_splits=3, n_repeats=5, shuffle=True, random_state=1)
    cvscores = []
    for train, test in kfold.split(X, Y):
        model = create_model()
        model.fit(X[train], Y[train], epochs=100, batch_size=50, verbose=1)
        scores = model.evaluate(X[test], Y[test], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
        
    return cvscores
 

In [194]:
def evaluate(seed, model, cv, mcf=False):
    print('1. Read files (seed={})'.format(seed))
       
    file35 = './data_family/seed{}_35perc.csv'.format(seed)
    file70='./data_family/seed{}_70perc.csv'.format(seed)
    file70HC= './data_family/seed{}_70percHC.csv'.format(seed)
    
    file35_GSE='./data_family/seed{}_35perc_gse109379.csv'.format(seed)
    file35_GSEHC ='./data_family/seed{}_35perc_HCgse109379.csv'.format(seed)
    file70_GSE = './data_family/seed{}_70perc_gse109379.csv'.format(seed)
    
    file70HC_GSE = './data_family/seed{}_70HC_gse109379.csv'.format(seed)
    file70_GSEHC = './data_family/seed{}_70perc_gse109379HC.csv'.format(seed)
    file70HC_GSEHC = './data_family/seed{}_70HC_gse109379HC.csv'.format(seed)
    
    
    fileHoldOut = './data_family/seed{}_holdOutTest.csv'.format(seed)
    #########
    print('\tReading', file35)
    X35, y35 = process_csv(file35)
    print('\tReading', file70)
    X70, y70 = process_csv(file70)
    print('\tReading', file70HC)
    X70HC, y70HC = process_csv(file70HC)
    
    print('\tReading', file35_GSE)
    X35gse, y35gse = process_csv(file35_GSE)
    print('\tReading', file35_GSEHC)
    X35gseHC, y35gseHC = process_csv(file35_GSEHC)
    print('\tReading', file70_GSE)
    X70gse, y70gse = process_csv(file70_GSE)
    
    print('\tReading', file70HC_GSE)
    X70HCgse, y70HCgse = process_csv(file70HC_GSE)
    print('\tReading', file70_GSEHC)
    X70gseHC, y70gseHC = process_csv(file70_GSEHC)
    print('\tReading', file70HC_GSEHC)
    XHC, yHC = process_csv(file70HC_GSEHC)
    
    print('\tReading', fileHoldOut)
    xitest, yitest = process_csv(fileHoldOut)
    ###################
    
    
    print('2. Evaluate model with 35% data')
    print('\tcross validate', model, 'with', cv)
    result_cv_35 = apply_CV(model, cv, X35, y35)
    print('\tvalidate against test set')
    result_ts_35 = evaluate_against_test_set(model, X35, y35, xitest, yitest)
          
    print('3. Evaluate model with 70% data')
    print('\tcross validate', model, 'with', cv)
    result_cv_70 = apply_CV(model, cv, X70, y70)
    print('\tvalidate against test set')
    result_ts_70 = evaluate_against_test_set(model, X70, y70, xitest, yitest)

    print('4. Evaluate model with 70% HC')    
    print('\tcross validate', model, 'with', cv)
    result_cv_70HC = apply_CV(model, cv, X70HC, y70HC)
    print('\tvalidate against test set')
    result_ts_70HC = evaluate_against_test_set(model, X70HC, y70HC, xitest, yitest)
    
    #########
    
    print('5. Evaluate model with 35% data with GSE109379')
    print('\tcross validate', model, 'with', cv)
    result_cv_35gse = apply_CV(model, cv, X35gse, y35gse)
    print('\tvalidate against test set')
    result_ts_35gse = evaluate_against_test_set(model, X35gse, y35gse, xitest, yitest)
    
    print('5. Evaluate model with 35% data with High confident GSE109379')
    print('\tcross validate', model, 'with', cv)
    result_cv_35gseHC = apply_CV(model, cv, X35gseHC, y35gseHC)
    print('\tvalidate against test set')
    result_ts_35gseHC = evaluate_against_test_set(model, X35gseHC, y35gseHC, xitest, yitest)
    
    print('6. Evaluate model with 70% + GSE90496 pseudo-labeled data')    
    print('\tcross validate', model, 'with', cv)
    result_cv_70gse = apply_CV(model, cv, X70gse, y70gse)
    print('\tvalidate against test set')
    result_ts_70gse = evaluate_against_test_set(model, X70gse, y70gse, xitest, yitest)
    
    ##########    
             
    print('7. Evaluate model with 70% HC + GSE109379')
    print('\tcross validate', model, 'with', cv)
    result_cv_70HCgse = apply_CV(model, cv, X70HCgse, y70HCgse)
    print('\tvalidate against test set')
    result_ts_70HCgse = evaluate_against_test_set(model, X70HCgse, y70HCgse, xitest, yitest)

    print('8. Evaluate model with 70% + HC GSE90496 ')    
    print('\tcross validate', model, 'with', cv)
    result_cv_70gseHC = apply_CV(model, cv, X70gseHC, y70gseHC)
    print('\tvalidate against test set')
    result_ts_70gseHC = evaluate_against_test_set(model, X70gseHC, y70gseHC, xitest, yitest)
      
    print('9. Evaluate model with HC pseudo-labeled data')    
    print('\tcross validate', model, 'with', cv)
    result_cv_HC = apply_CV(model, cv, XHC, yHC)
    print('\tvalidate against test set')
    result_ts_HC = evaluate_against_test_set(model, XHC, yHC, xitest, yitest)

    
    print('6. Store results.')
    df = pd.DataFrame(data=[], columns=['Seed','Dataset','Validation','Metric','Value'])
    df = append_cv_results(df, result_cv_35, seed, '35')
    df = append_testset_results(df, result_ts_35, seed, '35')
    
    df = append_cv_results(df, result_cv_70, seed, '70')
    df = append_testset_results(df, result_ts_70, seed, '70')
    
    df = append_cv_results(df, result_cv_70HC, seed, '70HC')
    df = append_testset_results(df, result_ts_70HC, seed, '70HC')
    
    df = append_cv_results(df, result_cv_35gse, seed, '35_GSE')
    df = append_testset_results(df, result_ts_35gse, seed, '35_GSE')
    
    df = append_cv_results(df, result_cv_35gseHC, seed, '35_gseHC')
    df = append_testset_results(df, result_ts_35gseHC, seed, '35_gseHC')
    
    df = append_cv_results(df, result_cv_70gse, seed, '70_GSE')
    df = append_testset_results(df, result_ts_70gse, seed, '70_GSE')
    
    df = append_cv_results(df, result_cv_70HCgse, seed, '70HC_GSE')
    df = append_testset_results(df, result_ts_70HCgse, seed, '70HC_GSE')
    df = append_cv_results(df, result_cv_70gseHC, seed, '70_gseHC')
    df = append_testset_results(df, result_ts_70gseHC, seed, '70_gseHC')
    df = append_cv_results(df, result_cv_HC, seed, 'HC')
    df = append_testset_results(df, result_ts_HC, seed, 'HC')
    
    #df_class_acc = pd.DataFrame(columns = ['Seed', 'Dataset'])
    
    df_class = create_per_class_acc_df(result_ts_35[3], seed, '35')
    
    df_class = df_class.append(create_per_class_acc_df(result_ts_70[3], seed, '70'))
    df_class = df_class.append(create_per_class_acc_df(result_ts_70HC[3], seed, '70HC'))
    
    df_class = df_class.append(create_per_class_acc_df(result_ts_35gse[3], seed, '35GSE'))
    df_class = df_class.append(create_per_class_acc_df(result_ts_35gseHC[3], seed, '35_gseHC'))
    df_class = df_class.append(create_per_class_acc_df(result_ts_70gse[3], seed, '70_GSE'))
    
    df_class = df_class.append(create_per_class_acc_df(result_ts_70HCgse[3], seed, '70HC_GSE'))
    df_class = df_class.append(create_per_class_acc_df(result_ts_70gseHC[3], seed, '70_gseHC'))
    df_class = df_class.append(create_per_class_acc_df(result_ts_HC[3], seed, 'HC'))
    
    return (df, df_class)



In [4]:
def count_pseudo_label_per_family(seed):
    print('1. Read files (seed={})'.format(seed))
       
    file35 = './data_family/seed{}_35perc.csv'.format(seed)
    file70='./data_family/seed{}_70perc.csv'.format(seed)
    file70HC= './data_family/seed{}_70percHC.csv'.format(seed)
    
    file35_GSE='./data_family/seed{}_35perc_gse109379.csv'.format(seed)
    file35_GSEHC ='./data_family/seed{}_35perc_HCgse109379.csv'.format(seed)
    file70_GSE = './data_family/seed{}_70perc_gse109379.csv'.format(seed)
    
    file70HC_GSE = './data_family/seed{}_70HC_gse109379.csv'.format(seed)
    file70_GSEHC = './data_family/seed{}_70perc_gse109379HC.csv'.format(seed)
    file70HC_GSEHC = './data_family/seed{}_70HC_gse109379HC.csv'.format(seed)
    
    fileHoldOut = './data_family/seed{}_holdOutTest.csv'.format(seed)
    
    #########
    
    print('\tReading', file35)
    X35, y35 = process_csv(file35)
    f35 = get_freq_series(y35)
    print('\tReading', file70)
    X70, y70 = process_csv(file70)
    f70 = get_freq_series(y70)
    print('\tReading', file70HC)
    X70HC, y70HC = process_csv(file70HC)
    f70HC = get_freq_series(y70HC)
    
    print('\tReading', file35_GSE)
    X35gse, y35gse = process_csv(file35_GSE)
    f35gse = get_freq_series(y35gse)
    print('\tReading', file35_GSEHC)
    X35gseHC, y35gseHC = process_csv(file35_GSEHC)
    f35gseHC = get_freq_series(y35gseHC)
    print('\tReading', file70_GSE)
    X70gse, y70gse = process_csv(file70_GSE)
    f70gse = get_freq_series(y70gse)
    
    print('\tReading', file70HC_GSE)
    X70HCgse, y70HCgse = process_csv(file70HC_GSE)
    f70HCgse = get_freq_series(y70HCgse)
    print('\tReading', file70_GSEHC)
    X70gseHC, y70gseHC = process_csv(file70_GSEHC)
    f70gseHC = get_freq_series(y70gseHC)
    print('\tReading', file70HC_GSEHC)
    XHC, yHC = process_csv(file70HC_GSEHC)
    fHC = get_freq_series(yHC)
    
    print('\tReading', fileHoldOut)
    xitest, yitest = process_csv(fileHoldOut)
    fitest = get_freq_series(yitest)
    
 
    print('2. Store pseudo_label_counts')
    df = pandas.DataFrame(data=[], columns = f35.keys())
    df = df.append(f35, ignore_index=True)
    df = df.append(f70, ignore_index=True)  
    df = df.append(f70HC, ignore_index=True)
    df = df.append(f35gse, ignore_index=True)
    df = df.append(f35gseHC, ignore_index=True)
    df = df.append(f70gse, ignore_index=True)
    df = df.append(f70HCgse, ignore_index=True)
    df = df.append(f70gseHC, ignore_index=True)
    df = df.append(fHC, ignore_index=True)
    df = df.append(fitest, ignore_index=True)
    
    df.insert(0, 'Dataset', ['35', '70', '70HC', '35gse', '35gseHC', '70gse', '70HCgse', '70gseHC', 'HC', 'holdOut'])
    df.insert(1, 'Seed', [seed]*10)
    
    return df


def combine_input_label2(seed):
    print('1. Read files (seed={})'.format(seed))
       
    file35 = './data_family/seed{}_35perc.csv'.format(seed)
    file70='./data_family/seed{}_70perc.csv'.format(seed)
    file70HC= './data_family/seed{}_70percHC.csv'.format(seed)
    
    file35_GSE='./data_family/seed{}_35perc_gse109379.csv'.format(seed)
    file35_GSEHC ='./data_family/seed{}_35perc_HCgse109379.csv'.format(seed)
    file70_GSE = './data_family/seed{}_70perc_gse109379.csv'.format(seed)
    
    file70HC_GSE = './data_family/seed{}_70HC_gse109379.csv'.format(seed)
    file70_GSEHC = './data_family/seed{}_70perc_gse109379HC.csv'.format(seed)
    file70HC_GSEHC = './data_family/seed{}_70HC_gse109379HC.csv'.format(seed)
    
    
    fileHoldOut = './data_family/seed{}_holdOutTest.csv'.format(seed)
    #########
    
    #########
    
    print('\tReading', file35)
    X35, y35 = process_csv(file35)
    f35 = get_freq_series(y35)
    print('\tReading', file70)
    X70, y70 = process_csv(file70)
    f70 = get_freq_series(y70)
    print('\tReading', file70HC)
    X70HC, y70HC = process_csv(file70HC)
    f70HC = get_freq_series(y70HC)
    
    print('\tReading', file35_GSE)
    X35gse, y35gse = process_csv(file35_GSE)
    f35gse = get_freq_series(y35gse)
    print('\tReading', file35_GSEHC)
    X35gseHC, y35gseHC = process_csv(file35_GSEHC)
    f35gseHC = get_freq_series(y35gseHC)
    print('\tReading', file70_GSE)
    X70gse, y70gse = process_csv(file70_GSE)
    f70gse = get_freq_series(y70gse)
    
    print('\tReading', file70HC_GSE)
    X70HCgse, y70HCgse = process_csv(file70HC_GSE)
    f70HCgse = get_freq_series(y70HCgse)
    print('\tReading', file70_GSEHC)
    X70gseHC, y70gseHC = process_csv(file70_GSEHC)
    f70gseHC = get_freq_series(y70gseHC)
    print('\tReading', file70HC_GSEHC)
    XHC, yHC = process_csv(file70HC_GSEHC)
    fHC = get_freq_series(yHC)
    
    print('\tReading', fileHoldOut)
    xitest, yitest = process_csv(fileHoldOut)
    fitest = get_freq_series(yitest)
    
 
    print('2. Store pseudo_label_counts')
    df = pandas.DataFrame(data=[], columns = f35.keys())
    df = df.append(f35, ignore_index=True)
    df = df.append(f70, ignore_index=True)  
    df = df.append(f70HC, ignore_index=True)
    df = df.append(f35gse, ignore_index=True)
    df = df.append(f35gseHC, ignore_index=True)
    df = df.append(f70gse, ignore_index=True)
    df = df.append(f70HCgse, ignore_index=True)
    df = df.append(f70gseHC, ignore_index=True)
    df = df.append(fHC, ignore_index=True)
    df = df.append(fitest, ignore_index=True)
    
    df.insert(0, 'Dataset', ['35', '70', '70HC', '35gse', '35gseHC', '70gse', '70HCgse', '70gseHC', 'HC', 'holdOut'])
    df.insert(1, 'Seed', [seed]*10)
    
    return df

In [ ]:
def evaluate_short(seed, model, cv, mcf=False):
    print('1. Read files (seed={})'.format(seed))
       
    file35 = './data_family/seed{}_35perc.csv'.format(seed)
    file70='./data_family/seed{}_70perc.csv'.format(seed)
    file70HC= './data_family/seed{}_70percHC.csv'.format(seed)
    
    
    
    fileHoldOut = './data_family/seed{}_holdOutTest.csv'.format(seed)
    #########
    print('\tReading', file35)
    X35, y35 = process_csv(file35)
    y35_one_hot = keras.utils.to_categorical(y35, num_classes)
    print('\tReading', file70)
    X70, y70 = process_csv(file70)
    print('\tReading', file70HC)
    X70HC, y70HC = process_csv(file70HC)
    
        
    print('\tReading', fileHoldOut)
    xitest, yitest = process_csv(fileHoldOut)
    ###################
    
    
    print('2. Evaluate model with 35% data')
    print('\tcross validate', model, 'with', cv)
    result_cv_35 = apply_CV(model, cv, X35, y35)
    print('\tvalidate against test set')
    result_ts_35 = evaluate_against_test_set(model, X35, y35, xitest, yitest)
          
    print('3. Evaluate model with 70% data')
    print('\tcross validate', model, 'with', cv)
    result_cv_70 = apply_CV(model, cv, X70, y70)
    print('\tvalidate against test set')
    result_ts_70 = evaluate_against_test_set(model, X70, y70, xitest, yitest)

    
    #########
    
        
    print('6. Store results.')
    df = pd.DataFrame(data=[], columns=['Seed','Dataset','Validation','Metric','Value'])
    df = append_cv_results(df, result_cv_35, seed, '35')
    df = append_testset_results(df, result_ts_35, seed, '35')
    
    df = append_cv_results(df, result_cv_70, seed, '70')
    df = append_testset_results(df, result_ts_70, seed, '70')
    
    
    #df_class_acc = pd.DataFrame(columns = ['Seed', 'Dataset'])
    
    df_class_acc = create_per_class_acc_df(result_ts_35[3], seed, '35')
    df_class_acc = df_class_acc.append(create_per_class_acc_df(result_ts_70[3], seed, '70'))
       
    return (df, df_class_acc)

In [ ]:

def run_all_seeds(rand):
    model = RandomForestClassifier(n_estimators = 50, max_depth=100, random_state=rand)
    cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=5)
    for seed in [1, 2, 20, 40, 80, 160, 320]:
        result = evaluate(seed, model, cv)
        output_file = 'result_family_seed{}_GSE109379_{}.csv'.format(seed, rand)
        output_per_family_file = './output_acc_perFamily/result_acc_perFamily_seed{}_GSE109379_{}.csv'.format(seed, rand)
        result[0].to_csv(output_file, index=False)
        result[1].to_csv(output_per_family_file, index=False)
        print('Result saved to', output_file)

def run_one_seeds(rand):
    model = RandomForestClassifier(n_estimators = 50, max_depth=100, random_state=rand)
    cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1)
    for seed in [ 2]:
        result = evaluate_short(seed, model, cv)
        output_file = './output/result_family_seed{}_GSE109379_{}.csv'.format(seed, rand)
        output_per_family_file = './output_acc_perFamily/result_acc_perFamily_seed{}_GSE109379_{}.csv'.format(seed, rand)
        result[0].to_csv(output_file, index=False)
        result[1].to_csv(output_per_family_file, index=False)
        print('Result saved to', output_file)
        
def create_pseudo_labels_file():
    for seed in [1, 2, 20, 40, 80, 160, 320]:
        combined_labels = count_pseudo_label_per_family(seed)
        output_file = 'pseudo_label_family_seed{}.csv'.format(seed)
        combined_labels.to_csv(output_file, index=True)

In [133]:
seed = 1
file35 = './data_family/seed{}_35perc.csv'.format(seed)    
fileHoldOut = './data_family/seed{}_holdOutTest.csv'.format(seed)
file35_GSE='./data_family/seed{}_35perc_gse109379.csv'.format(seed)


num_classes = 75

df = pd.read_csv(file35_GSE, index_col=0)


#########


In [102]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

# one hot encode
encoded = to_categorical(integer_encoded)
print(encoded)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [127]:
print('\tReading', file35)
X35, y35 = process_csv_encoder(file35)
print('\tReading', fileHoldOut)
xitest, yitest = process_csv_encoder(fileHoldOut)



	Reading ./data_family/seed1_35perc.csv
	Reading ./data_family/seed1_holdOutTest.csv


In [179]:

# split into input (X) and output (Y) variables
X = df.drop(columns=['y'])
integer_encoded = label_encoder.fit_transform(df.y)
y = keras.utils.to_categorical(integer_encoded)

print(X.shape, y.shape)

(2077, 5072) (2077, 75)


In [195]:

model = create_model(5072)
model.fit(X,y, epochs=10, batch_size=50, verbose=1)

Epoch 1/10
42/42 [==============================] - 1s 21ms/step - loss: 4.5249 - accuracy: 0.3665 - precision: 0.5925 - recall: 0.2802 - mae: 0.0191 - mse: 0.0108
Epoch 2/10
42/42 [==============================] - 1s 21ms/step - loss: 0.8870 - accuracy: 0.7905 - precision: 0.9317 - recall: 0.7111 - mae: 0.0081 - mse: 0.0037
Epoch 3/10
42/42 [==============================] - 1s 21ms/step - loss: 0.4735 - accuracy: 0.8813 - precision: 0.9723 - recall: 0.8127 - mae: 0.0055 - mse: 0.0023
Epoch 4/10
42/42 [==============================] - 1s 21ms/step - loss: 0.2857 - accuracy: 0.9238 - precision: 0.9690 - recall: 0.8849 - mae: 0.0037 - mse: 0.0015
Epoch 5/10
42/42 [==============================] - 1s 21ms/step - loss: 0.2426 - accuracy: 0.9314 - precision: 0.9706 - recall: 0.9044 - mae: 0.0033 - mse: 0.0013
Epoch 6/10
42/42 [==============================] - 1s 21ms/step - loss: 0.1687 - accuracy: 0.9488 - precision: 0.9741 - recall: 0.9250 - mae: 0.0024 - mse: 0.0010
Epoch 7/10
42/42

In [199]:
#bmod = baseline_model(5072)
#kfold = StratifiedKFold(n_splits=3, shuffle=True,random_state=1)
#estimator = KerasClassifier(build_fn=create_model(5072), epochs=10, batch_size=50, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_validate(model, X, y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f8ffcb08b20> does not.

In [200]:
cvscores = []
for train, test in kfold.split(X):
    model = create_model(5072)
    print(y[train])
    model.fit(X[train],y[train], epochs=10, batch_size=50, verbose=1)
    scores = model.evaluate(X[test], y[test], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


KeyError: "None of [Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,\n            ...\n            2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076],\n           dtype='int64', length=1869)] are in the [columns]"

In [148]:
n_inputs = X35.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(2500, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
model.add(keras.layers.Dense(1000, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
model.add(keras.layers.Dense(500, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
model.add(keras.layers.Dense(75, input_dim=n_inputs, activation='softmax'))



model.compile(optimizer = 'adam', loss= 'categorical_crossentropy', 
                  metrics=['accuracy', 'Precision','Recall', 'mae', 'mse'])



In [149]:
mhist = model.fit(X, y, epochs=5, batch_size=50)
cvscores = model.evaluate()


Epoch 1/5


ValueError: in user code:

    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:788 run_step  **
 

In [110]:
cvscores = model.evaluate(x_val, y_val, verbose=0)
cvscores.append(cvscores)



labels_p = model.predict(test_x)

y_predict = np.argmax(labels_p, axis=1)



NameError: name 'x_val' is not defined

In [22]:
run_one_seeds(123456)

1. Read files (seed=2)
	Reading ./data_family/seed2_35perc.csv
	Reading ./data_family/seed2_70perc.csv
	Reading ./data_family/seed2_70percHC.csv
	Reading ./data_family/seed2_holdOutTest.csv
2. Evaluate model with 35% data
	cross validate RandomForestClassifier(max_depth=100, n_estimators=50, random_state=123456) with RepeatedStratifiedKFold(n_repeats=1, n_splits=3, random_state=None)
	validate against test set


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


3. Evaluate model with 70% data
	cross validate RandomForestClassifier(max_depth=100, n_estimators=50, random_state=123456) with RepeatedStratifiedKFold(n_repeats=1, n_splits=3, random_state=None)
	validate against test set
6. Store results.
(0.919, 0.919, 0.923, {'ANA PA': 0.6666666666666666, 'CHGL': 0.5, 'CHORDM': 0.3333333333333333, 'CN': 1.0, 'CNS NB, FOXR2': 1.0, 'CONTR, ADENOPIT': 1.0, 'CONTR, CEBM': 0.5, 'CONTR, HEMI': 0.75, 'CONTR, HYPTHAL': 1.0, 'CONTR, INFLAM': 1.0, 'CONTR, PINEAL': 1.0, 'CONTR, PONS': 1.0, 'CONTR, REACT': 0.5714285714285714, 'CONTR, WM': 1.0, 'CPH, ADM': 1.0, 'CPH, PAP': 1.0, 'DLGNT': 0.0, 'DMG, K27': 1.0, 'EFT, CIC': 0.75, 'EPN, MPE': 1.0, 'EPN, PF A': 1.0, 'EPN, PF B': 0.8666666666666667, 'EPN, RELA': 1.0, 'EPN, SPINE': 0.75, 'EPN, YAP': 1.0, 'ETMR': 1.0, 'EWS': 0.5, 'GBM, G34': 0.5833333333333334, 'HGNET, BCOR': 1.0, 'HGNET, MN1': 0.6666666666666666, 'HMB': 1.0, 'IHG': 0.3333333333333333, 'LGG, DIG DIA': 1.0, 'LGG, DNT': 1.0, 'LGG, GG': 0.6666666666666666

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [24]:
run_all_seeds(123456)

1. Read files (seed=1)
	Reading ./data_family/seed1_35perc.csv
	Reading ./data_family/seed1_70perc.csv
	Reading ./data_family/seed1_70percHC.csv
	Reading ./data_family/seed1_35perc_gse109379.csv
	Reading ./data_family/seed1_35perc_HCgse109379.csv
	Reading ./data_family/seed1_70perc_gse109379.csv
	Reading ./data_family/seed1_70HC_gse109379.csv
	Reading ./data_family/seed1_70perc_gse109379HC.csv
	Reading ./data_family/seed1_70HC_gse109379HC.csv
	Reading ./data_family/seed1_holdOutTest.csv
2. Evaluate model with 35% data
	cross validate RandomForestClassifier(max_depth=100, n_estimators=50, random_state=123456) with RepeatedStratifiedKFold(n_repeats=5, n_splits=3, random_state=None)
	validate against test set


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


3. Evaluate model with 70% data
	cross validate RandomForestClassifier(max_depth=100, n_estimators=50, random_state=123456) with RepeatedStratifiedKFold(n_repeats=5, n_splits=3, random_state=None)
	validate against test set


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


4. Evaluate model with 70% HC
	cross validate RandomForestClassifier(max_depth=100, n_estimators=50, random_state=123456) with RepeatedStratifiedKFold(n_repeats=5, n_splits=3, random_state=None)
	validate against test set


...

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


8. Evaluate model with 70% + HC GSE90496 
	cross validate RandomForestClassifier(max_depth=100, n_estimators=50, random_state=123456) with RepeatedStratifiedKFold(n_repeats=5, n_splits=3, random_state=None)
	validate against test set


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


9. Evaluate model with HC pseudo-labeled data
	cross validate RandomForestClassifier(max_depth=100, n_estimators=50, random_state=123456) with RepeatedStratifiedKFold(n_repeats=5, n_splits=3, random_state=None)
	validate against test set
6. Store results.
Result saved to result_family_seed320_GSE109379_123456.csv


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [3]:
file1 = './data/seed2_35perc_train.Mat'
file2='./data/seed2_70perc_SSLabel.csv'
file3='./data/seed2_SSLabel_HC.csv'
X35, y35, xitest, yitest = read_Mat(file1)
X70, y70 = process_csv(file2)
XHC, yHC = process_csv(file3)

In [7]:
result35 = apply_CV(model, cv, X35, y35)
result_ts_35 = evaluate_against_test_set(model, X35, y35, xitest, yitest)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [8]:
result70 = apply_CV(model, cv, X70, y70)
result_ts_70 = evaluate_against_test_set(model, X70, y70, xitest, yitest)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [9]:
resultHC = apply_CV(model, cv, XHC, yHC)
result_ts_HC = evaluate_against_test_set(model, XHC, yHC, xitest, yitest)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [10]:
result_df = pandas.DataFrame(data=[], columns=['Seed','Dataset','Validation','Metric','Value'])

In [11]:
result_df = append_cv_results(result_df, result35, 2, '35%')
result_df = append_testset_results(result_df, result_ts_35, 2, '35%')
result_df = append_cv_results(result_df, result70, 2, '70%')
result_df = append_testset_results(result_df, result_ts_70, 2, '70%')
result_df = append_cv_results(result_df, resultHC, 2, 'HC')
result_df = append_testset_results(result_df, result_ts_HC, 2, 'HC')

In [15]:
result_df

,Seed,Dataset,Validation,Metric,Value
0,2,35%,cross_val,balanced_acc,0.710
1,2,35%,cross_val,f1,0.810
2,2,35%,cross_val,roc_auc_ovr,0.990
3,2,35%,vs_testset,prec,0.860
4,2,35%,vs_testset,rec,0.880
5,2,35%,vs_testset,acc,0.880
6,2,35%,vs_testset,balanced_acc,0.903
7,2,70%,cross_val,balanced_acc,0.830
8,2,70%,cross_val,f1,0.900
9,2,70%,cross_val,roc_auc_ovr,1.000
